# Assignment 1: N-Gram Language Models
**Group 15:** Anil Lingala (akl180001), Kenneth Ly (kll200003), Anshuman Das (axd210123), Parisa Nawar (pxn210032)  
**CS 6320.001**  
**09/07/25**  



# Setup

In [ ]:
import re
import math
import copy

# Dataset

You are given (via eLearning) a corpus of reviews of Chicago hotels Each line in these files corresponds to a single review. Here is an example:  

```
After Leaving some important documents in the room, I called and asked for the lost and
found department... SEVEN TIMES over the course of a week. Eventually I had enough and
asked for a manager and was put on hold then disconnected. Finally , a deceivingly friendly
operator PROMISED me she would have someone call me back in a few minutes, It never
happened. So Avoid this place because after they rip you off once they’ll keep doing it later.
They cannot be trusted at their word.
```
**\*Note for Google Colab you wil have to upload train.txt and val.txt to session storage every time you start a new session\***

In [ ]:
#Open the files and read them
trainFile = "train.txt"
valFile = "val.txt"

with open(trainFile, "r", encoding="utf-8") as f:
    trainCorpus = [line.lower().strip() for line in f]

with open(valFile, "r", encoding="utf-8") as f:
    valCorpus = [line.lower().strip() for line in f]

print("Sample training review:", trainCorpus[0])

Sample training review: i booked two rooms four months in advance at the talbott . we were placed on the top floor next to the elevators , which are used all night long . when speaking to the front desk , i was told that they were simply honoring my request for an upper floor , which i had requested for a better view . i am looking at a brick wall , and getting no sleep . he also told me that they had received complaints before from guests on the 16th floor , and were aware of the noise problem . why then did they place us on this floor when the hotel is not totally booked ? a request for an upper floor does not constitute placing someone on the top floor and using that request to justify this . if you decide to stay here , request a room on a lower floor and away from the elevator ! i spoke at length when booking my two rooms about my preferences . this is simply poor treatment of a guest whom they believed would not complain .


# Preprocessing Decisions and Helper Functions


Preprocessing The files included are already tokenized and hence it should be straightforward to obtain the tokens by using space as the delimiter. Feel free to do any other preprocessing that you might think is important for this corpus. Do not forget to describe and explain your pre-processing choices in your report.

In [ ]:
#Clean each train review and split it individually
def cleanLines(corpus):
  cleanedLines = []
  for line in corpus:
    line = re.sub(r'[^\w\s]', '', line)
    line = line.split(' ')

    while '' in line:
          line.remove('')

    while '\n' in line:
          line.remove('\n')

    line.insert(0, '<s>')
    line.append('</stop>')
    cleanedLines.append(line)
  return cleanedLines


#Helper function to sort dictionaries for printing
def sort(dict):
  return sorted(dict.items(), key=lambda x: x[1], reverse=True)

#Helper function to ensure that  the calculated probabilites of the models add up to 1
def validateProbabilties(dict):
  total_prob = sum(list(dict.values()))
  print("\nTotal probability:", total_prob)

In [ ]:
cleanTrainLines = cleanLines(trainCorpus)
cleanTestLines = cleanLines(valCorpus)
print("trainLines: ", cleanTrainLines[0])
print("testLines: ", cleanTestLines[0])

trainLines:  ['<s>', 'i', 'booked', 'two', 'rooms', 'four', 'months', 'in', 'advance', 'at', 'the', 'talbott', 'we', 'were', 'placed', 'on', 'the', 'top', 'floor', 'next', 'to', 'the', 'elevators', 'which', 'are', 'used', 'all', 'night', 'long', 'when', 'speaking', 'to', 'the', 'front', 'desk', 'i', 'was', 'told', 'that', 'they', 'were', 'simply', 'honoring', 'my', 'request', 'for', 'an', 'upper', 'floor', 'which', 'i', 'had', 'requested', 'for', 'a', 'better', 'view', 'i', 'am', 'looking', 'at', 'a', 'brick', 'wall', 'and', 'getting', 'no', 'sleep', 'he', 'also', 'told', 'me', 'that', 'they', 'had', 'received', 'complaints', 'before', 'from', 'guests', 'on', 'the', '16th', 'floor', 'and', 'were', 'aware', 'of', 'the', 'noise', 'problem', 'why', 'then', 'did', 'they', 'place', 'us', 'on', 'this', 'floor', 'when', 'the', 'hotel', 'is', 'not', 'totally', 'booked', 'a', 'request', 'for', 'an', 'upper', 'floor', 'does', 'not', 'constitute', 'placing', 'someone', 'on', 'the', 'top', 'floor'

# Unsmoothed Unigram Model

You must write the code for gathering n-gram counts and computing n-gram probabilities yourself. For example, consider the simple corpus consisting of the sole sentence:  

```
the students like the assignment
```
Part of what your program would compute for a unigram and bigram model, for example, would be the following:  

```
P (the) = 0.4, P (like) = 0.2
P (the|like) = 1.0, P (students|the) = 0.5
```

In [ ]:
#Create word dictionary and count total words for unigram
def countUnigrams(corpus):
  unigramCount = {}
  for line in corpus:
      for word in line:
          if word not in unigramCount:
              unigramCount[word] = 1
          else:
              unigramCount[word] += 1
  return unigramCount

baseUnigramCount = countUnigrams(cleanTrainLines)
totalUnigrams = sum(baseUnigramCount.values())
# print("Start unigram: ", baseUnigramCount["<s>"])
# print("Stop unigram: ", baseUnigramCount["</stop>"])
print("Top 5 unigram counts: ",sort(baseUnigramCount)[:5])
print("Total unigrams in corpus: ",totalUnigrams)



Top 5 unigram counts:  [('the', 5302), ('and', 2593), ('a', 2247), ('to', 2090), ('was', 1826)]
Total unigrams in corpus:  80423


In [ ]:
#Calculate unigram probabilities on training data
unigramModel = {}
for word, count in baseUnigramCount.items():
    probability = count / totalUnigrams
    unigramModel[word] = probability

print("Top 5 unigram probabilities: ",sort(unigramModel)[:5])

Top 5 unigram probabilities:  [('the', 0.0659264140855228), ('and', 0.03224202031757084), ('a', 0.027939768474192706), ('to', 0.025987590614625168), ('was', 0.022704947589619884)]


# Unsmoothed Bigram Model

In [ ]:
#Create word dictionary and count total words for bigram

from operator import itemgetter

def countBigrams(corpus):
  bigramCount = {}
  for line in corpus:
      for i in range(len(line) - 1):
          if (line[i], line[i + 1]) not in bigramCount:
              bigramCount[(line[i], line[i + 1])] = 1
          else:
              bigramCount[(line[i], line[i + 1])] += 1
  return bigramCount

baseBigramCount = countBigrams(cleanTrainLines)
totalBigrams = sum(baseBigramCount.values())

# print("Bigrams starting with <s>: ", sorted([(item[0], item[1]) for item in list(baseBigramCount.items()) if "<s>" in item[0]], key = itemgetter(1), reverse=True))
# print("Bigrams starting with </stop>: ", sorted([(item[0], item[1]) for item in list(baseBigramCount.items()) if "</stop>" in item[0]], key = itemgetter(1), reverse=True))
print("Top 5 bigram counts: ", sort(baseBigramCount)[:5])
print("Total bigrams in corpus: ", totalBigrams)

Top 5 bigram counts:  [(('in', 'the'), 420), (('the', 'hotel'), 414), (('of', 'the'), 343), (('at', 'the'), 335), (('the', 'room'), 295)]
Total bigrams in corpus:  79911


In [ ]:
#Calculate bigram probabilities
bigramModel = {}

for word, count in baseBigramCount.items():
    probability = count / (baseUnigramCount[word[0]])
    bigramModel[word] = probability

print("bigram probabilities: ", list(bigramModel.items())[:5])

bigram probabilities:  [(('<s>', 'i'), 0.216796875), (('i', 'booked'), 0.012273524254821741), (('booked', 'two'), 0.011627906976744186), (('two', 'rooms'), 0.0234375), (('rooms', 'four'), 0.0049504950495049506)]


# Smoothing and Unknown Words

Firstly, you should implement one or more than one methods to handle unknown words. Then You will need to implement two smoothing methods (e.g. Laplace, Add-k smoothing with different k). Teams can choose any method(s) that they want for each. The report should make clear what methods were selected, providing a description for any non-standard approach (e.g., an approach that was not covered in class or in the class)

# Unknown Words

In [ ]:
#Replaces every word that only appears n times in corpus with <unk>
def unknownWords(corpus, n):
  unkCorpus = copy.deepcopy(corpus)
  for line in unkCorpus:
      for i in range(len(line)):
          tok = line[i]
          if baseUnigramCount.get(line[i], 0) <= n:
              line[i] = '<unk>'
  return unkCorpus


In [ ]:
unkTrainLines = unknownWords(cleanTrainLines, 1)

unkUnigramCount = countUnigrams(unkTrainLines)
print("Top 5 unigram counts: ",sort(unkUnigramCount)[:5])

unkBigramCount = countBigrams(unkTrainLines)
print("Top 5 bigram counts: ", sort(unkBigramCount)[:5])

Top 5 unigram counts:  [('the', 5302), ('<unk>', 3113), ('and', 2593), ('a', 2247), ('to', 2090)]
Top 5 bigram counts:  [(('in', 'the'), 420), (('the', 'hotel'), 414), (('of', 'the'), 343), (('at', 'the'), 335), (('the', 'room'), 295)]


# Smoothing

In [ ]:
# Implement unigram smoothing method
def unigramSmoothing(ngramCounts, totalNgrams, vocabSize, k):
    smoothedModel = {}
    for ngram, count in ngramCounts.items():
        prob = (count + k) / (totalNgrams + (k * vocabSize))
        smoothedModel[ngram] = prob
    return smoothedModel

# Implement bigram smoothing method
def bigramSmoothing(ngramCounts, totalNgrams, vocabSize, k):
    smoothedModel = {}
    for ngram, count in ngramCounts.items():
        prob = (count + k) / (unkUnigramCount[ngram[0]] + (k * vocabSize))
        smoothedModel[ngram] = prob
    return smoothedModel

## Unigram Smoothing

In [ ]:
# Apply Laplace and add-K smoothing for unigram model
laplaceSmoothedUnigramModel = unigramSmoothing(unkUnigramCount, totalUnigrams, len(unkUnigramCount), 1)
k5SmoothedUnigramModel = unigramSmoothing(unkUnigramCount, totalUnigrams, len(unkUnigramCount), 5)

print("Laplace Smoothed Unigram Model:", sort(laplaceSmoothedUnigramModel)[:5])
print("K=5 Smoothed Unigram Model:", sort(k5SmoothedUnigramModel)[:5])

Laplace Smoothed Unigram Model: [('the', 0.06350746089914014), ('<unk>', 0.037292519939642166), ('and', 0.031065124188642188), ('a', 0.02692151086201528), ('to', 0.025041316375655674)]
K=5 Smoothed Unigram Model: [('the', 0.0553862531048446), ('<unk>', 0.03254085871130685), ('and', 0.027113903441942016), ('a', 0.02350289089732618), ('to', 0.021864367864075644)]


## Bigram Smoothing

In [ ]:
#Include the 0-count n-grams in the bigramCount word dictionary
words = list(unkUnigramCount.keys())
print("Total Unique Bigrams without 0-count: ", len(unkBigramCount))

for i in range(len(words)):
    for j in range(len(words)):
        pair = (words[i], words[j])
        if pair not in unkBigramCount:
          unkBigramCount[pair] = 0

print("Total Unique Bigrams with 0-count: ", len(unkBigramCount))

Total Unique Bigrams without 0-count:  33266
Total Unique Bigrams with 0-count:  9480241


In [ ]:
#Apply Laplace and add-K smoothing for bigram model
laplaceSmoothedBigramModel = bigramSmoothing(unkBigramCount, totalBigrams, len(unkUnigramCount), 1)
k5SmoothedBigramModel = bigramSmoothing(unkBigramCount, totalBigrams, len(unkUnigramCount), 5)

print("Laplace Smoothed Bigram Model: ", sort(laplaceSmoothedBigramModel)[:5])
print("K=5 Smoothed Bigram Model: ", sort(k5SmoothedBigramModel)[:5])

Laplace Smoothed Bigram Model:  [(('in', 'the'), 0.09702696473841899), (('at', 'the'), 0.08786610878661087), (('of', 'the'), 0.0833939393939394), (('on', 'the'), 0.06265125033611185), (('this', 'hotel'), 0.05717397222978492)]
K=5 Smoothed Bigram Model:  [(('in', 'the'), 0.025517862503752625), (('of', 'the'), 0.021166595705857306), (('at', 'the'), 0.021065675340768277), (('the', 'hotel'), 0.020244479876310575), (('and', 'the'), 0.015899488547920837)]


## Perplexity Calculation  

Implement code to compute the perplexity of a “development set.” (“development set” is just another way to refer to the validation set — part of a dataset that is distinct from the training portion.) Compute and report the perplexity of your model (with variations) on it. Under the second definition above, perplexity is a function of the average (per-word) log probability: use this to avoid numerical computation errors.  

If you experimented with more than one type of smoothing and unknown word handling, you should report and compare the perplexity results of experiments among some of them

In [ ]:
# Function to compute unigram perplexity
def unigramPerplexity(valCleanLines, unigramCounts, k):
  wordCount, log_sum=0, 0.0
  vSize = len(unigramCounts)
  unkUniSum = sum(unigramCounts.values())

  for read in valCleanLines:
    for currentWord in read:
      if currentWord in ('<s>', '</stop>'):
          continue

      curr = unigramCounts.get(currentWord, 0)
      prob = (curr + k) / (unkUniSum + k * vSize)
      log_sum += math.log(prob)
      wordCount += 1

  if wordCount == 0:
        raise ValueError("No valid unigram token found after skipping.")
  return math.exp(-log_sum/wordCount)

In [ ]:
# Function to compute bigram perplexity
def bigramPerplexity(valCleanLines, unigramCounts, bigramCounts, k):
  wordCount, log_sum=0, 0.0
  vSize = len(unigramCounts)
  unkBigSum = sum(bigramCounts.values())

  for read in valCleanLines:
    for i in range(1, len(read)):
      prevWord = read[i-1]
      currentWord = read[i]
      if prevWord in ('<s>', '</stop>') or currentWord in ('<s>', '</stop>'):
          continue

      prevC = unigramCounts.get(prevWord, 0)
      bigCount = bigramCounts.get((prevWord, currentWord), 0)

      denominator = prevC + k * vSize
      p = (bigCount + k) / (denominator if denominator > 0 else k * vSize)
      log_sum += math.log(p)
      wordCount += 1

  if wordCount == 0:
        raise ValueError("No valid bigram token found after skipping.")
  return math.exp(-log_sum/wordCount)

In [ ]:
unkTrainLines = unknownWords(cleanTrainLines, 1)
unkTestLines  = unknownWords(cleanTestLines, 1)
unkUnigramCount = countUnigrams(unkTrainLines)
unkBigramCount  = countBigrams(unkTrainLines)

# Unigram perplexities
print("Unigram Perplexity (Laplace k=1):", unigramPerplexity(unkTestLines, unkUnigramCount, 1.0))

print("Unigram Perplexity (Add-k k=5):", unigramPerplexity(unkTestLines, unkUnigramCount, 5.0))

# Bigram perplexities
print("Bigram Perplexity (Laplace k=1):", bigramPerplexity(unkTestLines, unkUnigramCount, unkBigramCount, 1.0))

print("Bigram Perplexity (Add-k k=5):", bigramPerplexity(unkTestLines, unkUnigramCount, unkBigramCount, 5.0))

Unigram Perplexity (Laplace k=1): 290.73172953869454
Unigram Perplexity (Add-k k=5): 299.5746749358359
Bigram Perplexity (Laplace k=1): 374.6149788575254
Bigram Perplexity (Add-k k=5): 736.996281527752


In [ ]:
# Unigram perplexities with training set
print("Unigram Perplexity Training (Unsmoothed k=0):",
      unigramPerplexity(unkTrainLines, unkUnigramCount, 0.0))

print("Unigram Perplexity Training (Laplace k=1):",
      unigramPerplexity(unkTrainLines, unkUnigramCount, 1.0))

print("Unigram Perplexity Training (Add-k k=5):",
      unigramPerplexity(unkTrainLines, unkUnigramCount, 5.0))

print("Unigram Perplexity Training (Add-k k=0.01):",
      unigramPerplexity(unkTrainLines, unkUnigramCount, 0.01))


# Unigram perplexities with validation set
print("Unigram Perplexity (Unsmoothed k=0):",
      unigramPerplexity(unkTestLines, unkUnigramCount, 0.0))

print("Unigram Perplexity (Laplace k=1):",
      unigramPerplexity(unkTestLines, unkUnigramCount, 1.0))

print("Unigram Perplexity (Add-k k=5):",
      unigramPerplexity(unkTestLines, unkUnigramCount, 5))

print("Unigram Perplexity (Add-k k=0.01):",
      unigramPerplexity(unkTestLines, unkUnigramCount, 0.01))

# Bigram perplexities
print("Bigram Perplexity Training (Laplace k=1):",
      bigramPerplexity(unkTrainLines, unkUnigramCount, unkBigramCount, 1.0))

print("Bigram Perplexity Training (Add-k k=5):",
      bigramPerplexity(unkTrainLines, unkUnigramCount, unkBigramCount, 5.0))

print("Bigram Perplexity Training (Add-k k=0.01):",
      bigramPerplexity(unkTrainLines, unkUnigramCount, unkBigramCount, 0.01))

print("Bigram Perplexity (Laplace k=1):",
      bigramPerplexity(unkTestLines, unkUnigramCount, unkBigramCount, 1.0))

print("Bigram Perplexity (Add-k k=5):",
      bigramPerplexity(unkTestLines, unkUnigramCount, unkBigramCount, 5.0))

print("Bigram Perplexity (Add-k k=0.01):",
      bigramPerplexity(unkTestLines, unkUnigramCount, unkBigramCount, 0.01))

Unigram Perplexity Training (Unsmoothed k=0): 386.11772158789967
Unigram Perplexity Training (Laplace k=1): 387.20980605600755
Unigram Perplexity Training (Add-k k=5): 402.5512281420322
Unigram Perplexity Training (Add-k k=0.01): 386.11607557700347
Unigram Perplexity (Unsmoothed k=0): 335.9370345355836
Unigram Perplexity (Laplace k=1): 338.7385266431256
Unigram Perplexity (Add-k k=5): 356.908884001531
Unigram Perplexity (Add-k k=0.01): 335.95702789316056
Bigram Perplexity Training (Laplace k=1): 459.00977621500067
Bigram Perplexity Training (Add-k k=5): 1077.4830558111491
Bigram Perplexity Training (Add-k k=0.01): 52.39193778150466
Bigram Perplexity (Laplace k=1): 543.6087164017083
Bigram Perplexity (Add-k k=5): 1095.2674933291883
Bigram Perplexity (Add-k k=0.01): 190.58533397139834
